# ETL Proccess for CENIPA

## 1.0 - Libraries and Functions

### 1.1 - Importing Libraries

In [94]:
# Requests
from requests import get

# Data manipulation
import pandas as pd
import pandera as pa
import datetime

### 1.2 - Building Helper Functions

In [4]:
def jupyter_settings():
    
    pd.options.display.max_columns = None
    pd.options.display.max_rows = 200
    pd.set_option('display.expand_frame_repr', False)
    
jupyter_settings()

## 2.0 - Data Extraction

In [52]:
# loading data by request url from open data government
url = 'http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia.csv'
dataset = pd.read_csv(url, sep=';')

# Dimension data
print(f'Number of Lines: {dataset.shape[0]}\nNumber of Columns: {dataset.shape[1]}\nNumber of Dimensionality: {dataset.shape[0] * dataset.shape[1]}')

Number of Lines: 6114
Number of Columns: 22
Number of Dimensionality: 134508


## 3.0 - Data Transformation

### 3.1 - Data Validation

In [53]:
# Copy data for security lost
validation = dataset.copy()

In [67]:
validation.head()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,BRASIL,****,03/01/2010,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,BRASIL,SBBE,03/01/2010,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,BRASIL,SBRJ,03/01/2010,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,BRASIL,****,04/01/2010,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,BRASIL,SBPK,05/01/2010,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


In [211]:
# Datatypes
## Concatenation data types with the 3 first lines to compare types of data with really data are
describe = pd.concat([validation.describe(include='all', datetime_is_numeric=True).T], axis=1)
describe = pd.concat([describe, validation.head(3).T], axis=1)

## For a global validation by schema pre-determinited you can use the pandera module
schema = pa.DataFrameSchema(
    columns = {
         'codigo_ocorrencia': pa.Column(pa.Int),
         'ocorrencia_classificacao': pa.Column(pa.String),
         'ocorrencia_latitude': pa.Column(pa.String, nullable=True),
         'ocorrencia_longitude': pa.Column(pa.String, nullable=True),
         'ocorrencia_cidade': pa.Column(pa.String),
         'ocorrencia_uf': pa.Column(pa.String, pa.Check.str_length(2, 2)),
         'ocorrencia_pais': pa.Column(pa.String),
         'ocorrencia_aerodromo': pa.Column(),
         #'ocorrencia_dia': pa.Column('O'),
         #'ocorrencia_hora': pa.Column(pa.String, pa.Check.str_matches(r'([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$')),
         #'investigacao_aeronave_liberada': pa.Column('O'),
         #'investigacao_status': pa.Column('O'),
         #'divulgacao_relatorio_numero': pa.Column('O'),
         #'divulgacao_relatorio_publicado': pa.Column('O'),
         #'divulgacao_dia_publicacao': pa.Column('O'),
         #'total_recomendacoes': pa.Column('int64'),
         #'total_aeronaves_envolvidas': pa.Column('int64'),
         #'ocorrencia_saida_pista': pa.Column('O')
    }

)
try:
    schema.validate(validation)
    print('validado')
except Exception as e:
    e.failure_cases['column_name'] = e.schema.name
    print(e)
    
try:
    errors_list_validation.append(error.failure_cases)
except NameError as e:
    errors_list_validation = list()
    errors_list_validation.append(error.failure_cases)
finally:
    errors_list_validation[0].to_parquet(f'dataset/errors_list_validation_{datetime.date.today()}.gzip', compression='gzip')
    
describe
# None a Number (NaN)
# Data inconsistency
## dropping duplicates columns ( all ocorrencias ) are equal
# columns_dropped_list = ['codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3', 'codigo_ocorrencia4']
## Descriptive analysis

<Schema Column(name=ocorrencia_uf, type=DataType(str))> failed element-wise validator 0:
<Check str_length: str_length(2, 2)>
failure cases:
   index failure_case
0   2227          ***
1   5855          ***


,count,unique,top,freq,mean,std,min,25%,50%,75%,max,0,1,2
codigo_ocorrencia,6114.0,NaN,NaN,NaN,59775.058063,14552.373693,39115.0,46586.25,53110.5,78142.5,80259.0,40211,40349,40351
codigo_ocorrencia1,6114.0,NaN,NaN,NaN,59775.058063,14552.373693,39115.0,46586.25,53110.5,78142.5,80259.0,40211,40349,40351
codigo_ocorrencia2,6114.0,NaN,NaN,NaN,59775.058063,14552.373693,39115.0,46586.25,53110.5,78142.5,80259.0,40211,40349,40351
codigo_ocorrencia3,6114.0,NaN,NaN,NaN,59775.058063,14552.373693,39115.0,46586.25,53110.5,78142.5,80259.0,40211,40349,40351
codigo_ocorrencia4,6114.0,NaN,NaN,NaN,59775.058063,14552.373693,39115.0,46586.25,53110.5,78142.5,80259.0,40211,40349,40351
ocorrencia_classificacao,6114,3,INCIDENTE,3393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INCIDENTE,INCIDENTE,INCIDENTE
ocorrencia_latitude,4552,2732,***,724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,***,NaN,NaN
ocorrencia_longitude,4552,2728,***,725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,***,NaN,NaN
ocorrencia_cidade,6114,1133,RIO DE JANEIRO,331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RIO DE JANEIRO,BELÉM,RIO DE JANEIRO
ocorrencia_uf,6114,28,SP,1464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RJ,PA,RJ


### 3.2 - Data Cleaning

### 3.3 - Data Preparation

## 4.0 - Data Load

## 5.0 - Production